In [1]:
!pip install transformers
!pip install evaluate
!pip install sentencepiece
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 14.7 MB/s eta 0:00:00


In [1]:
import torch
import re
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import pytorch_lightning as pl
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download('omw-1.4')
import os
import string
import torch.nn as nn
import pandas as pd
import numpy as np
import evaluate
import matplotlib.pyplot as plt
import transformers
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import get_linear_schedule_with_warmup, AdamW
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /home/aparna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aparna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/aparna/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
from datasets import load_dataset

ds = load_dataset("owkin/medical_knowledge_from_extracts")

In [2]:
from datasets import load_dataset

ds1 = load_dataset("ccdv/arxiv-summarization", "section")

In [3]:
ds1 = ds1["train"].shuffle(seed=42).select(range(len(ds1["train"]) // 8))

In [5]:
print(ds['train'].to_pandas().head(5))

                                            Question  \
0  Your goal is to extract structured information...   
1  Your goal is to extract structured information...   
2  Your goal is to extract structured information...   
3  Your goal is to extract structured information...   
4  Your goal is to extract structured information...   

                                              Answer  
0  {'conditions': 'Asthma', 'interventions': 'Beh...  
1  {'conditions': 'Hypertension', 'interventions'...  
2  {'conditions': 'Tardive Dyskinesia|Schizophren...  
3  {'conditions': 'Pulmonary Disease, Chronic Obs...  
4  {'conditions': 'Thrombosis, Venous|Acute Deep ...  


In [12]:
print(ds1.to_pandas().head())

                                             article  \
0  arp  220 is the nearest ( @xmath3 77  mpc ) ex...   
1  this study was supported by the danish nationa...   
2  the author expresses special thanks to h. hori...   
3  i zw 18 remains the most metal - poor blue com...   
4  several atoms play basic roles in modern physi...   

                                            abstract  
0  the cores of arp 220 , the closest ultralumino...  
1  in communication networks structure and dynami...  
2  the excited states in @xmath0 are investigated...  
3  long - slit keck ii , 4 m kitt peak , and 4.5 ...  
4  quantum electrodynamics is the first successfu...  


In [18]:
MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
BATCH_SIZE = 4
TEXT_LEN = 512
HEADLINE_LEN = 64
EPOCHS = 2
DEVICE = "cpu"

In [19]:
class SummaryDataset(Dataset):
    def __init__(self, ds1, tokenizer, text_len, headline_len):
        self.ds1 = ds1
        self.headlines = self.ds1["abstract"]
        self.text = self.ds1["article"]
        self.tokenizer = tokenizer
        self.text_len = text_len
        self.headline_len = headline_len
  
    def __len__(self):
        return len(self.headlines)
  
    def __getitem__(self, idx):
        # T5 transformers performs different tasks by prepending the particular prefix to the input text.
        text = "summarize:" + str(self.text[idx])                # In order to avoid dtype mismatch, as T5 is text-to-text transformer, the datatype must be string
        headline = str(self.headlines[idx])

        text_tokenizer = self.tokenizer(text, max_length=self.text_len, padding="max_length",
                                                        truncation=True, add_special_tokens=True)
        headline_tokenizer = self.tokenizer(headline, max_length=self.headline_len, padding="max_length",
                                                        truncation=True, add_special_tokens=True)
        return {
            "input_ids": torch.tensor(text_tokenizer["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(text_tokenizer["attention_mask"], dtype=torch.long),
            "summary_ids": torch.tensor(headline_tokenizer["input_ids"], dtype=torch.long),
            "summary_mask": torch.tensor(headline_tokenizer["attention_mask"], dtype=torch.long)
        }

In [20]:
class SummaryDataModule(pl.LightningDataModule):
    def __init__(self, 
                train_df, 
                val_df,
                test_df,
                batch_size,
                tokenizer,
                text_len,
                summary_len):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.batch_size = batch_size
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.text_len = text_len
        self.summary_len = summary_len
    
    def setup(self, stage=None):
        self.train_dataset = SummaryDataset(
            self.train_df,
            self.tokenizer,
            self.text_len,
            self.summary_len)
        
        self.val_dataset = SummaryDataset(
            self.val_df,
            self.tokenizer,
            self.text_len,
            self.summary_len)
        
        self.test_dataset = SummaryDataset(
            self.test_df,
            self.tokenizer,
            self.text_len,
            self.summary_len
        )
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=4)
        
    def test_dataloader(self):
        return DataLoader(
              self.test_dataset,
              batch_size=self.batch_size,
              num_workers=4
          )

In [24]:
df1 = ds1.to_pandas()
train_df, test_df = train_test_split(df1, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


In [25]:
summary_module = SummaryDataModule(train_df, val_df, test_df, BATCH_SIZE, TOKENIZER, TEXT_LEN, HEADLINE_LEN)
summary_module.setup()

In [26]:
class SummaryModel(pl.LightningModule):
    def __init__(self):
        super(SummaryModel, self).__init__()
        self.model = MODEL
  
    def forward(self, input_ids, attention_mask, labels=None, decoder_attention_mask=None):
        outputs = self.model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels,
                        decoder_attention_mask=decoder_attention_mask)
        return outputs.loss, outputs.logits
  
    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["summary_ids"]
        decoder_attention_mask = batch["summary_mask"]

        loss, output = self(input_ids, attention_mask, labels, decoder_attention_mask)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["summary_ids"]
        decoder_attention_mask = batch["summary_mask"]

        loss, output = self(input_ids, attention_mask, labels, decoder_attention_mask)
        return loss
 
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        loss, output = self(input_ids=input_ids, 
                      attention_mask=attention_mask)
        return loss
    
    def configure_optimizers(self):
        optimizer = AdamW(self.model.parameters(), lr=0.0001)
        scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=0,
                num_training_steps=EPOCHS*len(ds1))
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

In [27]:
model = SummaryModel()

In [29]:
trainer = pl.Trainer(
    max_epochs=EPOCHS,
)

trainer.fit(model, summary_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M  | eval
------------------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)
0         Modules in train mode
541       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
summary_model = SummaryModel.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
summary_model.freeze()

In [ ]:
def summarize(text):
    inputs = TOKENIZER(text, 
                       max_length=TEXT_LEN,
                       truncation=True,
                       padding="max_length",
                       add_special_tokens=True, 
                       return_tensors="pt")
    summarized_ids = summary_model.model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"], 
        num_beams=4)

    return " ".join([TOKENIZER.decode(token_ids, skip_special_tokens=True)
                    for token_ids in summarized_ids])

In [ ]:
bleu = evaluate.load("google_bleu")

In [1]:
df1 = ds1.to_pandas()
text = df1.iloc[100]["article"]
headline = df1.iloc[100]["abstract"]
summarized_text = summarize(text)

NameError: name 'ds1' is not defined

In [ ]:
bleu.compute(predictions=[summarized_text], references=[headline])